In [3]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [5]:
ANALYTICS_BASE_URL = "https://api.info.v2.tibetswap.io/"

In [6]:
base_url = ANALYTICS_BASE_URL

In [12]:
def make_url(base, verb, params):
    return base + verb + '?' + '&'.join([f'{k}={v}' for k, v in params.items()])

In [22]:
def get_launcher_id(token):
    # should do a lookup, but for now just an example
    if token=='STDG':
        return '9124c0198ed1109fdc3b41d193b67fe06135b82a3f00129d66650f47ff51151b'
    else:
        return NotImplemented

In [28]:
def get_transactions(token, lookback=7, op=None, limit=None):
    params = {}
    params['pair_launcher_id'] = get_launcher_id(token)
    if op is not None:
        params['operation'] = op
    if limit is not None:
        params['limit'] = limit
    url = make_url(base_url, 'transactions', params)
    response = requests.get(url)
    code = response.status_code
    if code != 200:
        return f'Failed with status code {code}'
    df = pd.json_normalize(response.json())
    df['time'] = pd.to_datetime(df.timestamp, unit='s')
    start = datetime.now() - timedelta(days=lookback)
    return df[df.time > start].copy()

In [36]:
def get_volume(token, lookback=7):
    df = get_transactions(token, lookback=lookback, op='SWAP')
    if type(df) == str:
        return df
    return df['state_change.xch'].abs().sum().astype('float') / 1e12

In [38]:
get_volume('STDG')

0.728811535696

In [39]:
get_transactions('STDG', op='SWAP')

,pair_launcher_id,operation,height,coin_id,timestamp,state_change.xch,state_change.token,state_change.liquidity,time
0,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3771037,da48e215f3b008bf5da4f30272da1a1dbcd0b660aa6002...,1686136123,200000000000,-2025017,0,2023-06-07 11:08:43
1,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3767064,3e9b740737c6a183dc54e2573e097dc3d210c072dfc4a2...,1686062718,-154143842754,1600000,0,2023-06-06 14:45:18
2,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3766871,4c6a176a3ba3c5a8296e182c11941b04928142a8d7c041...,1686059463,80000000000,-803282,0,2023-06-06 13:51:03
3,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3765260,f0d019a761686c57d803d3c1b7b07a46f8c80840ce47e8...,1686029849,-73552346166,750000,0,2023-06-06 05:37:29
4,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3752972,c2088b276a963ea6f4a794ec51317e03c298403c6aa86c...,1685801247,19999953832,-198374,0,2023-06-03 14:07:27
5,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3749305,64fced5e37f013f3370baea45a060526cde8e3f1a5eeca...,1685732253,4999995409,-49905,0,2023-06-02 18:57:33
6,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3745410,be659ff0c6a52a45f7e5d7f9942f8e81ab702ae052c876...,1685659341,99999932393,-1025250,0,2023-06-01 22:42:21
7,9124c0198ed1109fdc3b41d193b67fe06135b82a3f0012...,SWAP,3742651,9814ca86cc7ac763a9d9de66c8a967146b3dc27c52f6e5...,1685608538,-96115465142,1000000,0,2023-06-01 08:35:38
